In [1]:
from dotenv import load_dotenv
load_dotenv(".env.local")

True

In [2]:
from langchain_community.document_loaders.doc_intelligence import (
    AzureAIDocumentIntelligenceLoader,
)
import os
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
def load_docs_adi(filepath: str):
    adi = AzureAIDocumentIntelligenceLoader(
        "https://eastus.api.cognitive.microsoft.com",
        AZURE_DOCUMENT_INTELLIGENCE_API_KEY,
        file_path=filepath,
        api_model="prebuilt-read",
    )
    docs = adi.load()
    return docs

In [3]:
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "Service Center Student Employee Handbook.docx")

docs = load_docs_adi(file_path)

In [7]:
with open("content.txt", "w") as f:
    f.write(docs[0].page_content)

In [8]:
from langchain_core.documents import Document
with open("content.txt", "r") as f:
    content = f.read()
doc = Document(content)


In [9]:
doc

Document(metadata={}, page_content='Department of Residential Facilities\nDivision of Student Affairs\nService Center Student Employment Handbook\nTable of Contents\nDepartment of Residential Facilities Mission Statement\n3\nIntroduction to the Department of Residential Facilities\n4\nIntroduction to & Purpose of the Service Center\n4\nGeneral Expectations of Desk Associates\n5 - 7\nWork Breaks & Lunch / Dinner Periods\n8\nReporting to Work, Lateness & Substitutes\n8 - 9\nParking While Working at the Service Center\n9 - 11\nInclement Weather\n11\nUniversity ID Cards\n11\nAccidents or Injury on the Job\n12\nSafe and Sick Leave\n12\nAlcohol / Drug Use Policy\n13\nSexual Misconduct Policy\n13 - 14\nEmployment Records\n14\nService Center Employee Probationary Period\n14\nPerformance Evaluation\n15\nUnsatisfactory Performance & Discipline\n16 - 17\nAppeal Process\n16\nResignation\n16\nPriority Points\n16\nScheduling\n17 - 18\nPayroll\n19 - 20\nPaychecks & Paycheck Distribution\n21\nSummer B